### Data Normalization

In [ ]:

import os
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image

# Directory Paths
AUGMENT_DIR = r"C:\Users\USER\Documents\Thesis Dataset\Processed Dataset\train_augmented"
VAL_DIR     = r"C:\Users\USER\Documents\Thesis Dataset\Processed Dataset\val"
TEST_DIR    = r"C:\Users\USER\Documents\Thesis Dataset\Processed Dataset\test"
IMG_EXTENSIONS = (".jpg", ".jpeg", ".png")

# Parameters for Normalization
IMAGENET_MEAN = [0.485, 0.456, 0.406]
IMAGENET_STD = [0.229, 0.224, 0.225]
IMG_SIZE = 224

# Data Transfroms
train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomRotation(15),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

val_test_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=IMAGENET_MEAN, std=IMAGENET_STD)
])

# Printing the Datasets and its classes
class PlantDiseaseDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.class_to_idx = {}
        
        class_idx = 0
        for crop_type in sorted(os.listdir(data_dir)):
            crop_path = os.path.join(data_dir, crop_type)
            if not os.path.isdir(crop_path):
                continue
                
            for disease_class in sorted(os.listdir(crop_path)):
                class_path = os.path.join(crop_path, disease_class)
                if not os.path.isdir(class_path):
                    continue
                
                full_class_name = f"{crop_type}_{disease_class}"
                if full_class_name not in self.class_to_idx:
                    self.class_to_idx[full_class_name] = class_idx
                    class_idx += 1
                
                for img_file in os.listdir(class_path):
                    if img_file.lower().endswith(IMG_EXTENSIONS):
                        img_path = os.path.join(class_path, img_file)
                        self.images.append(img_path)
                        self.labels.append(self.class_to_idx[full_class_name])
        
        print(f"Dataset loaded: {len(self.images)} images, {len(self.class_to_idx)} classes")
        print(f"Classes: {list(self.class_to_idx.keys())}")
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

# Normalization Phase
print("🔄 Creating normalized datasets...")

train_dataset = PlantDiseaseDataset(AUGMENT_DIR, transform=train_transforms)
val_dataset = PlantDiseaseDataset(VAL_DIR, transform=val_test_transforms)
test_dataset = PlantDiseaseDataset(TEST_DIR, transform=val_test_transforms)

BATCH_SIZE = 32

train_loader = DataLoader(
    train_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=True, 
    num_workers=4,
    pin_memory=True if torch.cuda.is_available() else False
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=4,
    pin_memory=True if torch.cuda.is_available() else False
)

test_loader = DataLoader(
    test_dataset, 
    batch_size=BATCH_SIZE, 
    shuffle=False, 
    num_workers=4,
    pin_memory=True if torch.cuda.is_available() else False
)

print("✅ Normalization setup complete!")
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Number of classes: {len(train_dataset.class_to_idx)}")
print(f"Batch size: {BATCH_SIZE}")

# Verify whether the normalization works
def verify_normalization():
    """Verify that normalization is applied correctly"""
    sample_batch = next(iter(train_loader))
    images, labels = sample_batch
    
    print("\n📊 Normalization Verification:")
    print(f"Image tensor shape: {images.shape}")
    print(f"Image tensor range: [{images.min():.3f}, {images.max():.3f}]")
    print(f"Mean per channel: {images.mean(dim=[0,2,3])}")
    print(f"Std per channel: {images.std(dim=[0,2,3])}")
    print("✅ If values are close to mean≈[0,0,0] and std≈[1,1,1], normalization is working!")

verify_normalization()


🔄 Creating normalized datasets...
Dataset loaded: 66156 images, 18 classes
Classes: ['Banana_Leaf_Disease_Dataset_Bangladesh_Healthy', 'Banana_Leaf_Disease_Dataset_Bangladesh_Panama Disease', 'Banana_Leaf_Disease_Dataset_Bangladesh_Yellow and Black Sigatoka', 'Banana_Leaf_Disease_Dataset_Bangladesh_cordana', 'Banana_Leaf_Disease_Dataset_Bangladesh_pestalotiopsis', 'Coconut Tree Disease Dataset_Bud Root Dropping', 'Coconut Tree Disease Dataset_Bud Rot', 'Coconut Tree Disease Dataset_Gray Leaf Spot', 'Coconut Tree Disease Dataset_Healthy_Leaves', 'Coconut Tree Disease Dataset_Leaf Rot', 'Coconut Tree Disease Dataset_WCLWD_DryingofLeaflets', 'Coconut Tree Disease Dataset_WCLWD_Flaccidity', 'Coconut Tree Disease Dataset_WCLWD_Yellowing', 'Sugarcane Leaf Disease Dataset_Healthy', 'Sugarcane Leaf Disease Dataset_Mosaic', 'Sugarcane Leaf Disease Dataset_RedRot', 'Sugarcane Leaf Disease Dataset_Rust', 'Sugarcane Leaf Disease Dataset_Yellow']
Dataset loaded: 2365 images, 18 classes
Classes: ['B

### Train CNN-ViT

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from timm import create_model

# ---- CONFIG ----
AUGMENT_DIR = r"C:\Users\USER\Documents\Thesis Dataset\Processed Dataset\train_augmented"
VAL_DIR     = r"C:\Users\USER\Documents\Thesis Dataset\Processed Dataset\val"
TEST_DIR    = r"C:\Users\USER\Documents\Thesis Dataset\Processed Dataset\test"
IMG_EXTENSIONS = (".jpg", ".jpeg", ".png")
BATCH_SIZE = 32
NUM_CLASSES = 18

# ---- DATA LOADING ----
data_transforms = {  
    'train': transforms.Compose([transforms.ToTensor()]),
    'val': transforms.Compose([transforms.ToTensor()]),
}

train_dataset = datasets.ImageFolder(AUGMENT_DIR, transform=data_transforms['train'])
val_dataset = datasets.ImageFolder(VAL_DIR, transform=data_transforms['val'])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

# ---- MODEL LOADING ----
def get_resnet50(num_classes):
    model = models.resnet50(weights='IMAGENET1K_V2')
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

def get_efficientnet_b0(num_classes):
    model = models.efficientnet_b0(weights='IMAGENET1K_V1')
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    return model

def get_vit_small(num_classes):
    model = create_model("vit_small_patch16_224", pretrained=True, num_classes=num_classes)
    return model

models_dict = {
    'resnet50': get_resnet50(NUM_CLASSES),
    'efficientnet_b0': get_efficientnet_b0(NUM_CLASSES),
    'vit_small': get_vit_small(NUM_CLASSES)
}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
for name in models_dict:
    models_dict[name] = models_dict[name].to(device)

# ---- TRAINING ----
def train_one_epoch(model, dataloader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, targets in dataloader:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    return running_loss / total, correct / total

def validate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, targets in dataloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            running_loss += loss.item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return running_loss / total, correct / total

NUM_EPOCHS = 25
for model_name, model in models_dict.items():
    print(f"\nTraining {model_name}...")
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    best_val_acc = 0.0
    for epoch in range(NUM_EPOCHS):
        train_loss, train_acc = train_one_epoch(model, train_loader, criterion, optimizer)
        val_loss, val_acc = validate(model, val_loader, criterion)
        print(f"Epoch {epoch+1}/{NUM_EPOCHS} - Train Acc: {train_acc:.4f} - Val Acc: {val_acc:.4f}")
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), f"{model_name}_best.pth")
    print(f"Finished training {model_name} with best val acc: {best_val_acc:.4f}")

# ---- ENSEMBLE PREDICTION EXAMPLE ----
def ensemble_predict(models_dict, inputs, weights=None):
    models = [model.eval() for model in models_dict.values()]
    if weights is None:
        weights = [1/len(models)] * len(models)
    logits = [model(inputs.to(device)) * w for model, w in zip(models, weights)]
    return torch.sum(torch.stack(logits), dim=0)
